In [6]:
#Enter the location of the QUALTX output file
QUALTXoutfile = r'C:\Users\Ernest\Ernest_Sandbox\TurboQUALTX\TurboQUALTX\RB55NOV.OUT'

In [7]:
#Import python packages
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import os
import ET_Utils.QUALTX_Utils
import numpy as np
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [8]:
#Parse the QUALTX output file 
Scenario = QUALTXoutfile
DOstds = [5,4.8]
WQC_of_interest = ['DO_MG/L','BOD_MG/L','NH3_MG/L','NO3+2_MG/L','PHOS_MG/L']
StreamNames, Rdf, Hdf, AllWQdf = ET_Utils.QUALTX_Utils.Process_QUALTX(QUALTXoutfile,
                                                                      Scenario,DOstds = DOstds,
                                                                      WQC_of_interest = WQC_of_interest,
                                                                      loc = 1,plot_pdf = 0)

PECAN_LITTLE ELM CR
OUTFALL 001 COVE
OUTFALL 002 TRIB


In [9]:
#Perform plotting
x_range = [max(AllWQdf['ENDING_DIST']),min(AllWQdf['ENDING_DIST'])]

y_label = 'Concentration (mg/L)'
x_label = 'Distance (km)'

#Determine range of y-axis
maxy = []
for tempWQC in WQC_of_interest:
    maxy.append(max(AllWQdf[tempWQC]))
y_range = [0,round(max(maxy)*1.5)]

#Set line width
lws = np.zeros(len(WQC_of_interest))+2

#Set line colors
plot_colors = ['blue','black','green','cyan','orange']

#Set line symbols
plot_symbols = ['-','-','-','-','-']

#Set marker sizes
markersizes = np.zeros(len(WQC_of_interest))+1

#Set label sizes
x_labelsize = 15
y_labelsize = 15 

In [10]:
# Loop through each stream and plot the WQ profile plot
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

#output_file("legend.html", title="legend.py example")

#show(vplot(p1, p2))  # open a browser

for StreamName in StreamNames:
    p = figure(title=StreamName, tools=TOOLS,x_range = x_range,y_range=y_range,plot_width=900, plot_height=450)  
    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label
    WQdf = AllWQdf[AllWQdf['Stream']==StreamName].copy()
    SRdf = Rdf[Rdf['StreamName']==StreamName].reset_index()
    
    for k in range(0,len(WQC_of_interest)):        
        #ls.append(ax.plot(np.asarray(WQdf['ENDING_DIST']),np.asarray(WQdf[WQC_of_interest[k]]),plot_symbols[k], 
        #lw = lws[k],markersize=markersizes[k],
        #                  color = plot_colors[k],label = WQC_of_interest[k]))
        # add a line renderer
        p.line(np.asarray(WQdf['ENDING_DIST']),np.asarray(WQdf[WQC_of_interest[k]]), 
               legend=WQC_of_interest[k], line_width= lws[k],
               color = plot_colors[k],name = WQC_of_interest[k])

         #Overplot reach start and end points of each reach
        if len(SRdf) > 0:
            Points1 = list(SRdf['BEGIN NAME'])
            Rkms1 = list(SRdf['BEGIN REACH KM'])
            if len(SRdf) > 1:
                Points2 = [SRdf['END NAME'][SRdf.index[-1]]]
                Rkms2 = [SRdf['END REACH KM'][SRdf.index[-1]]]
            else:
                Points2 = list(SRdf['END NAME'])
                Rkms2 = list(SRdf['END REACH KM'])
                
            Points = Points1 + Points2
        
            #Rkms = Rkms1.append(Rkms2)
            Rkms = Rkms1+Rkms2
        
            for i in range(0,len(Points)):
                #print Points[i]
                p.line([Rkms[i],Rkms[i]],y_range,'--',color = 'k')
                p.text(Rkms[i],np.mean(y_range),Points[i])#,text_align = 'center')#,angle = 3.14159265/2)
                

        if len(DOstds) > 0:
            for DOstd in DOstds:
                p.line(x_range,[DOstd,DOstd],'--',color = 'k')
                p.text(np.mean(x_range),DOstd,"DO std = "+"{:4.1f}".format(DOstd)+" mg/L")

        
    show(p)